In [15]:
import requests
from requests_oauthlib import OAuth2Session
from io import StringIO
import nbformat
import chardet
import re

In [32]:
client_id = ""
client_secret = ""
authorization_base_url = 'https://github.com/login/oauth/authorize'
token_url = 'https://github.com/login/oauth/access_token'
redirect_uri = 'https://localhost:8000/callback'
block_size = 2048

In [33]:
def get_github_access_token(client_id, client_secret, authorization_response):
    github = OAuth2Session(client_id, redirect_uri=redirect_uri)
    token = github.fetch_token(token_url, authorization_response=authorization_response, client_secret=client_secret)
    return token['access_token']

In [34]:
def extract_python_code_from_ipynb(ipynb_content):
    notebook = nbformat.reads(ipynb_content.getvalue(), as_version=4)
    code_cells = [cell['source'] for cell in notebook['cells'] if cell['cell_type'] == 'code']
    return '\n'.join(code_cells)

In [35]:
def get_repo_contents(repo_url, path='', access_token=None):
    _, username, repo_name = repo_url.rstrip('/').rsplit('/', 2)
    user_input = f"I will provide you code for the repository {repo_name} in the next prompts, keep it in chat history"
    main_list = []
    main_list.append(user_input)

    api_url = f"https://api.github.com/repos/{username}/{repo_name}/contents/{path}"

    headers = {}
    if access_token:
        headers['Authorization'] = f"Bearer {access_token}"

    response = requests.get(api_url, headers=headers)

    if response.status_code == 200:
        contents = response.json()
        
        print(contents)

        for i in contents:
            if i:
                if 'type' in i and i['type'] == 'file' and i['name'].lower().endswith(('.py', '.js', '.java', '.c', '.cpp', '.cs', '.html', '.css', '.rb', '.php', '.swift', '.m', '.sql', '.sh', '.ts', '.go', '.rs', '.kt', '.vue', '.jsx', '.ipynb')):
                    file_content_url = i['download_url']
                    file_response = requests.get(file_content_url, headers=headers)

                    if file_response.status_code == 200:
                        content_input = f"This is the code for {i['name']}"
                        main_list.append(content_input)
                        if i['name'].lower().endswith('.ipynb'):
                            ipynb_content = StringIO(file_response.content.decode('utf-8', errors='replace'))
                            python_code = extract_python_code_from_ipynb(ipynb_content)
                            while python_code:
                                block = python_code[:block_size]
                                block_input = block
                                main_list.append(block_input)
                                python_code = python_code[block_size:]
                        else:
                            encoding = chardet.detect(file_response.content)['encoding']
                            encoding = encoding if encoding else 'utf-8'
                            file_content = file_response.content.decode(encoding, errors='replace')
                            while file_content:
                                block = file_content[:block_size]
                                block_input = block
                                main_list.append(block_input)
                                file_content = file_content[block_size:]
                    elif 'type' in i and i['type'] == 'dir':
                        folder_input = f"This is content in folder{i}"
                        main_list.append(folder_input)
                        get_repo_contents(repo_url, path=f"{path}/{i}", access_token=access_token)
                        root_input = "Now we are back in the root directory of the repository"
                        main_list.append(root_input)
        s1 = f"This is all the pieces of code I have for {repo_name}, now I will ask you questions about {repo_name}"
        main_list.append(s1)
    else:
        print(f"Failed to fetch repository contents. Status code: {response.status_code}")

    return main_list

In [36]:
def get_repo_list(access_token):
    headers = {'Authorization': f'Bearer {access_token}'}
    response = requests.get('https://api.github.com/user/repos', headers=headers)
    if response.status_code == 200:
        print(response.json())
        return response.json()
    else:
        print(f"Failed to fetch repository list. Status code: {response.status_code}")
        return None

In [ ]:
github_username = input("Enter your GitHub username: ")

In [ ]:
github = OAuth2Session(client_id, redirect_uri=redirect_uri, scope=['repo'])
authorization_url, state = github.authorization_url(authorization_base_url)
print(f'Please go to {authorization_url} and authorize access.')



In [ ]:
authorization_response = input('Paste the full redirect URL here: ')
token = get_github_access_token(client_id, client_secret, authorization_response)

repo_list = get_repo_list(token)

print(repo_list)

In [ ]:
if repo_list:
    print("Repositories:")
    for repo in repo_list:
        print(repo['name'])
    print()

    selected_repo = input("Enter the name of the repository you want to access: ")

    u = get_repo_contents(f"https://github.com/{github_username}/{selected_repo}", access_token=token)
    print(u)